In [91]:
import pandas as pd
import os
import shutil
from datetime import datetime

In [92]:
shift_dir = "./../../data/4_shift/"
rem_i_dir = "./../../data/5_remove_invalid/"

In [93]:
shift_dir = "./../../data/7point8_c_detrended/wavelet/"
rem_i_dir = "./../../data/10point6_c_remove_invalid/wavelet/"

Create invalid-valid table

In [94]:
all_dates = []
for fl in os.listdir(shift_dir):
    df = pd.read_csv(shift_dir + fl, index_col=0)
    df.index = pd.to_datetime(df.index)
    all_dates.extend(df.index)

all_dates = list(set(all_dates))

date_df = pd.DataFrame(index=all_dates, columns=[name[:-4] for name in os.listdir(shift_dir)])

for date in all_dates:
    for n in os.listdir(shift_dir):
        df = pd.read_csv(shift_dir + n, index_col=0)
        df = df.dropna()
        df = df[(df >= 0).all(1)]
        df.index = pd.to_datetime(df.index)
        if date in df.index:
            date_df.loc[date, n[:-4]] = 1
        else:
            date_df.loc[date, n[:-4]] = 0

date_df = date_df.sort_index()
date_df

/var/folders/5w/bchvy66s6vbgncfkws1skvf40000gp/T/ipykernel_51923/894866199.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)
/var/folders/5w/bchvy66s6vbgncfkws1skvf40000gp/T/ipykernel_51923/894866199.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)
/var/folders/5w/bchvy66s6vbgncfkws1skvf40000gp/T/ipykernel_51923/894866199.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index)
/var/folders/5w/bchvy66s6vbgncfkws1skvf40000gp/T/ipykernel_51923/894866199.py:4: UserWarnin

,CR,OE,INF,ROA,PR,NIA,NIM,RA,SCTA,CDR,LAS,SIZE
2018-01-01,0,0,0,0,0,0,0,0,0,0,0,0
2018-02-01,0,0,0,0,0,0,0,0,0,0,0,0
2018-03-01,0,0,0,0,1,0,0,0,0,0,0,0
2018-04-01,0,0,0,0,1,0,0,0,0,0,0,0
2018-05-01,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,0,1,1,0,0,1,1,0,0,0,1,1
2024-01-01,1,0,1,1,0,0,1,0,1,1,1,1
2024-02-01,1,1,1,0,0,1,0,1,0,1,0,0
2024-03-01,0,0,0,0,0,0,0,0,0,0,0,0


Delete nagative and zeros

In [95]:
negative_dates = set()

for fl in os.listdir(shift_dir):
    local_plus = []
    df = pd.read_csv(shift_dir + fl, index_col=0)
    df.index = pd.to_datetime(df.index, format = '%m/%Y').strftime('%m/%Y')
    for i in df.index:
        year_pos = True
        for v in df.loc[i]:
            if v < 0:
                year_pos = False
                break
        if year_pos:
            local_plus.append(i)
    if not negative_dates:
        negative_dates.update(local_plus)
    else:
        negative_dates = negative_dates.intersection(local_plus)
    
non_neg_dates = sorted(negative_dates, key = lambda x: datetime.strptime(x, '%m/%Y'))

In [96]:
for fl in os.listdir(shift_dir):
    local_plus = []
    df = pd.read_csv(shift_dir + fl, index_col=0)
    df_sel = df.loc[df.index.intersection(non_neg_dates)]
    df_sel.to_csv(rem_i_dir + fl)